In [1]:
from __future__ import print_function

Load in the Python modules necessary to load the data (gzip and yaml), process the data (numpy and pandas), and plot the data (toyplot).

In [2]:
import gzip
import yaml
import numpy
import pandas
import toyplot.pdf

print("yaml version:    ", yaml.__version__)
print("numpy version:   ", numpy.__version__)
print("pandas version:  ", pandas.__version__)
print("toyplot version: ", toyplot.__version__)

yaml version:     3.12
numpy version:    1.13.3
pandas version:   0.20.3
toyplot version:  0.16.0


## Ingest Data

Load the data, which is output into a YAML files.

In [3]:
filename = 'miniGraphics-skybridge-vn-scaling.yaml.gz'
yaml_data = yaml.load(gzip.open(filename))
data_vn = pandas.DataFrame(yaml_data)

filename = 'miniGraphics-skybridge-smp-scaling.yaml.gz'
yaml_data = yaml.load(gzip.open(filename))
data_smp = pandas.DataFrame(yaml_data)

Some of the original runs gave suspect results where the runs took much longer than expected. These appear to be transient issues in the system since a repeat of the run is closer to what is expected. To get more realistic results, I re-ran these conditions. So jettison these runs from the original data and load up the re-runs.

In [4]:
# Remove incorrect readings
data_vn = data_vn.loc[
    ((data_vn['composite-algorithm'] != '2-3-SwapBase') |
     (data_vn['num-processes'] != 4096) |
     (data_vn['image-height'] != 1080)) &
    ((data_vn['composite-algorithm'] != 'BinarySwapFold') |
     (data_vn['num-processes'] != 2896) |
     (data_vn['image-height'] != 1080)) &
    ((data_vn['composite-algorithm'] != 'BinarySwapRemainder') |
     (data_vn['num-processes'] != 2048) |
     (data_vn['image-height'] != 1080)) &
    ((data_vn['composite-algorithm'] != 'BinarySwapTelescoping') |
     (data_vn['num-processes'] != 1616) |
     (data_vn['image-height'] != 1080))
]
# Load correct readings and add them to the data
filename = 'miniGraphics-skybridge-vn-corrections.yaml.gz'
yaml_data = yaml.load(gzip.open(filename))
data_vn = pandas.concat([data_vn, pandas.DataFrame(yaml_data)], ignore_index=True)

The YAML data is hierarchical. The basic yaml reader to DataFrame just embeds dictionaries and lists in DataFrame columns. Fix that by expanding the data of these columns into new columns.

In [5]:
def expand_single_column(original_data, column_to_expand):

    expanded_data = pandas.DataFrame()
    for index in original_data.index:
        sub_table = pandas.DataFrame(original_data[column_to_expand][index])
        for column in original_data.columns:
            if column != column_to_expand:
                sub_table[column] = numpy.full(sub_table.index.shape,
                                               original_data[column][index],
                                               dtype=original_data[column].dtype)
        expanded_data = expanded_data.append(sub_table, ignore_index=True)
    return expanded_data

def flatten_table(original_data):
    flat_data = original_data
    for column_name in original_data.columns:
        if isinstance(flat_data[column_name][0], list):
            flat_data = expand_single_column(flat_data, column_name)
    return flat_data

In [6]:
data_vn = flatten_table(data_vn)
data_smp = flatten_table(data_smp)

Add a column that gives a human-readable name to each image resolution.

In [7]:
image_height_names = {
    500: 'Desktop Window',
    1080: 'HDTV',
    4320: '8K UHD',
}

data_vn['image-size'] = data_vn['image-height'].map(image_height_names)
data_smp['image-size'] = data_smp['image-height'].map(image_height_names)

Rename the algorithms from the identifiers the program wrote out to the strings used in the paper. Note that there are some extras in the data that we are ignoring.

In [8]:
algorithm_names = {
    '2-3-SwapBase': '2-3 Swap',
    'BinarySwapFold': 'Naive',
    'BinarySwapTelescoping': 'Telescoping',
    'BinarySwapRemainder': 'Remainder',
    'IceTBase': 'IceT'
}

data_vn['composite-algorithm'] = data_vn['composite-algorithm'].map(algorithm_names)
data_smp['composite-algorithm'] = data_smp['composite-algorithm'].map(algorithm_names)

Print a summary of the table data. There are multiple ways that Jupyter and pandas will report a summary of a table, but I find this method the most effective. It prints out every column. Then for all columns with a "small" number of unique values, it gives those values. This latter information really helps identify the proper way to group values.

In [9]:
import IPython.display

data_description = '##### Virtual Node\n\n'

for column_name in data_vn.columns:
    data_description = data_description + '**' + column_name + '**: '
    unique_values = data_vn[column_name].unique()
    if (len(unique_values) < 10):
        for value in unique_values:
            data_description = data_description + str(value) + ' '
    elif (numpy.issubdtype(unique_values.dtype, numpy.number)):
        data_description = (
            data_description +
            str(numpy.nanmin(unique_values)) + ' &ndash; ' +
            str(numpy.nanmax(unique_values)) + ' '
        )
    elif not pandas.isnull(unique_values).any():
        data_description = (
            data_description +
            str(numpy.min(unique_values)) + ' &ndash; ' +
            str(numpy.max(unique_values)) + ' '
        )
    data_description = data_description + ' \n'
    
data_description = data_description + '\n##### SMP\n\n'

for column_name in data_smp.columns:
    data_description = data_description + '**' + column_name + '**: '
    unique_values = data_smp[column_name].unique()
    if (len(unique_values) < 10):
        for value in unique_values:
            data_description = data_description + str(value) + ' '
    elif (numpy.issubdtype(unique_values.dtype, numpy.number)):
        data_description = (
            data_description +
            str(numpy.nanmin(unique_values)) + ' &ndash; ' +
            str(numpy.nanmax(unique_values)) + ' '
        )
    elif not pandas.isnull(unique_values).any():
        data_description = (
            data_description +
            str(numpy.min(unique_values)) + ' &ndash; ' +
            str(numpy.max(unique_values)) + ' '
        )
    data_description = data_description + ' \n'
    
IPython.display.display(IPython.display.Markdown(data_description))

##### Virtual Node

**color-buffer-format**: byte  
**composite-algorithm**: Naive Remainder Telescoping 2-3 Swap IceT nan  
**composite-seconds**: 0.00119348 &ndash; 3.49925  
**compress-seconds**:  
**construct-tree-seconds**: 0.000182052 &ndash; 0.0610641  
**depth-buffer-format**: float  
**gather-seconds**: 0.00071677 &ndash; 3.45673  
**geometry**: box  
**geometry-distribution**: duplicate  
**geometry-overlap**: -0.05  
**icet-copy-result-seconds**:  
**image-compression**: True False  
**image-height**: 288 1080 4320  
**image-width**: 512 1920 7680  
**num-processes**: 128 &ndash; 8192  
**num-triangles**: 1536 &ndash; 98304  
**paint-seconds**: 0.00011823 &ndash; 0.147793  
**painter**: simple  
**partial-composite-seconds**: 0.000293153 &ndash; 0.86969  
**phi-rotation**: -178.406 &ndash; 136.756  
**random-seed**: 17627  
**rendering-order-dependent**: False  
**start-time**: 2018-05-30T02:23:10.000000000 &ndash; 2018-06-08T12:10:45.000000000  
**theta-rotation**: -177.945 &ndash; 179.418  
**total-seconds**: 0.00146716 &ndash; 3.59942  
**trial-num**: 0 &ndash; 19  
**uncompress-seconds**:  
**zoom**: 1.5  
**image-size**: nan HDTV 8K UHD  

##### SMP

**color-buffer-format**: byte  
**composite-algorithm**: Naive Remainder Telescoping 2-3 Swap IceT nan  
**composite-seconds**: 0.000823865 &ndash; 0.99317  
**compress-seconds**:  
**construct-tree-seconds**: 1.1473e-05 &ndash; 0.00111429  
**depth-buffer-format**: float  
**gather-seconds**: 0.000536648 &ndash; 0.381332  
**geometry**: box  
**geometry-distribution**: duplicate  
**geometry-overlap**: -0.05  
**icet-copy-result-seconds**:  
**image-compression**: True False  
**image-height**: 288 1080 4320  
**image-width**: 512 1920 7680  
**num-processes**: 8 &ndash; 512  
**num-triangles**: 96 &ndash; 6144  
**paint-seconds**: 0.000120004 &ndash; 0.204059  
**painter**: simple  
**partial-composite-seconds**: 0.000219604 &ndash; 0.802574  
**phi-rotation**: -178.406 &ndash; 136.756  
**random-seed**: 17627  
**rendering-order-dependent**: False  
**start-time**: 2018-05-30T02:21:27.000000000 &ndash; 2018-06-08T12:10:16.000000000  
**theta-rotation**: -177.945 &ndash; 179.418  
**total-seconds**: 0.00107058 &ndash; 1.15555  
**trial-num**: 0 &ndash; 19  
**uncompress-seconds**:  
**zoom**: 1.5  
**image-size**: nan HDTV 8K UHD  


## Plot Data

We are plotting the time it takes to do a "partial composite" (that is the time to blend all the pixels, but the pixels are left distributed across all the processes).

The first thing we want to do is to average the time it took over all trials. This is easily done with a pivot table. We also need the filter the data to those that have been run in both VN and SMP modes. These are those between 64 and 512 processes.

In [10]:
filtered_indices = (
    (data_vn['num-processes'] >= 128) &
    (data_vn['num-processes'] <= 512)
)

average_partial_composite_vn = data_vn[filtered_indices].pivot_table(
    values='partial-composite-seconds',
    index='num-processes',
    columns=[
        'image-size',
        'composite-algorithm',
    ],
    aggfunc='mean',
)

average_partial_composite_vn

image-size             8K UHD                                            \
composite-algorithm  2-3 Swap      IceT     Naive Remainder Telescoping   
num-processes                                                             
128                  0.424882  0.053204  0.450411  0.415041    0.415397   
160                  0.440534  0.052022  0.685614  0.410063    0.412039   
176                  0.443978  0.053023  0.685982  0.412858    0.412648   
192                  0.438112  0.044091  0.679504  0.409229    0.409191   
224                  0.437661  0.041432  0.677950  0.406705    0.409481   
256                  0.414052  0.037369  0.402548  0.430229    0.402594   
272                  0.428487  0.038341  0.665274  0.403638    0.403873   
320                  0.426395  0.037734  0.668252  0.399939    0.400913   
352                  0.427184  0.037632  0.663289  0.396885    0.398925   
400                  0.425750  0.033612  0.661835  0.397030    0.397765   
448                  0.421872  0.031573  0.662567  0.402201    0.412633   
512                  0.407533  0.029525  0.395685  0.395133    0.394878   

image-size               HDTV                                            
composite-algorithm  2-3 Swap      IceT     Naive Remainder Telescoping  
num-processes                                                            
128                  0.023590  0.003565  0.022431  0.022514    0.022490  
160                  0.024490  0.004788  0.037664  0.022203    0.022332  
176                  0.023896  0.005028  0.037944  0.022328    0.022242  
192                  0.023896  0.004002  0.037442  0.022204    0.022245  
224                  0.024472  0.003871  0.037503  0.022121    0.022121  
256                  0.023289  0.003264  0.022116  0.021993    0.022151  
272                  0.023900  0.003878  0.036865  0.022126    0.022183  
320                  0.023573  0.002853  0.036933  0.021988    0.022029  
352                  0.023456  0.003754  0.036448  0.021906    0.021776  
400                  0.023836  0.002649  0.036566  0.021612    0.021997  
448                  0.023965  0.003078  0.036506  0.022167    0.021651  
512                  0.023806  0.002864  0.021664  0.021638    0.022055

In [11]:
filtered_indices = (
    (data_smp['num-processes'] >= 128) &
    (data_smp['num-processes'] <= 512)
)

average_partial_composite_smp = data_smp[filtered_indices].pivot_table(
    values='partial-composite-seconds',
    index='num-processes',
    columns=[
        'image-size',
        'composite-algorithm',
    ],
    aggfunc='mean',
)

average_partial_composite_smp

image-size             8K UHD                                            \
composite-algorithm  2-3 Swap      IceT     Naive Remainder Telescoping   
num-processes                                                             
128                  0.346160  0.025752  0.324867  0.325992    0.325512   
143                  0.383349  0.035832  0.620365  0.329196    0.330120   
161                  0.362933  0.037592  0.609298  0.335417    0.322732   
181                  0.350075  0.028211  0.601279  0.318414    0.318868   
203                  0.368279  0.025565  0.606522  0.320694    0.318968   
228                  0.359755  0.023898  0.605073  0.314026    0.314605   
256                  0.329589  0.017220  0.308654  0.308345    0.308671   
287                  0.359717  0.028142  0.596548  0.312132    0.315081   
322                  0.349128  0.029455  0.593704  0.310503    0.308433   
362                  0.339380  0.021054  0.589601  0.306951    0.307453   
406                  0.348666  0.018447  0.582892  0.304362    0.302856   
456                  0.343736  0.016096  0.583155  0.299269    0.300455   
512                  0.323190  0.012963  0.300264  0.300909    0.301313   

image-size               HDTV                                            
composite-algorithm  2-3 Swap      IceT     Naive Remainder Telescoping  
num-processes                                                            
128                  0.018828  0.001801  0.016950  0.017302    0.017308  
143                  0.020551  0.002746  0.031701  0.017456    0.017435  
161                  0.019846  0.002704  0.031150  0.017175    0.017202  
181                  0.019369  0.002439  0.031606  0.017061    0.017088  
203                  0.020347  0.002287  0.032032  0.017223    0.017191  
228                  0.020020  0.002169  0.031738  0.017156    0.016907  
256                  0.018597  0.001812  0.016653  0.016841    0.016982  
287                  0.020175  0.002269  0.031012  0.017163    0.017393  
322                  0.019464  0.002474  0.032058  0.016914    0.016995  
362                  0.019154  0.002034  0.032794  0.017007    0.016998  
406                  0.020316  0.001912  0.032517  0.016765    0.016890  
456                  0.019635  0.001807  0.032158  0.016752    0.016840  
512                  0.019172  0.001552  0.016727  0.016755    0.016832

In [12]:
numpy.max(numpy.max(average_partial_composite_vn['HDTV']))

0.037943829999999998

Make a grouping structure of the data so we can pull out the actual data values for each trial.

In [13]:
image_size = 'HDTV'

canvas = toyplot.Canvas('9.25in', '3in',
                        #style={'background-color': 'yellow'},
                       )

x_vn = average_partial_composite_vn.index
x_smp = average_partial_composite_smp.index

maxy = max(
    numpy.max(numpy.max(average_partial_composite_vn[image_size])),
    numpy.max(numpy.max(average_partial_composite_smp[image_size])),
)

xlocator = toyplot.locator.Log(base=2, format='{:.0f}')

def doPlot(axes, algorithm):
    axes.x.ticks.locator = xlocator
    axes.y.domain.min = 0
    axes.y.domain.max = maxy
    
    y_vn = numpy.array(average_partial_composite_vn[image_size, algorithm])
    y_smp = numpy.array(average_partial_composite_smp[image_size, algorithm])

    axes.plot(x_vn, y_vn)
    axes.plot(x_smp, y_smp)

    axes.text(
        512, numpy.max(y_vn), 'Virtual Node: ' + algorithm,
        style={
            'text-anchor': 'end',
            '-toyplot-vertical-align': 'last-baseline',
        }
    )
    axes.text(
        512, numpy.min(y_smp), 'Pure Distributed: ' + algorithm,
        style={
            'text-anchor': 'end',
            '-toyplot-vertical-align': 'top',
        }
    )

axesNaive = canvas.cartesian(
    xlabel = 'Number of Processes',
    ylabel = 'Partial Composite Time (seconds)',
    xscale='log',
    bounds=('5%', '33%', '5%', '85%'),
)
doPlot(axesNaive, 'Naive')

axes23 = canvas.cartesian(
    xlabel = 'Number of Processes',
    #ylabel = 'Partial Composite Time (seconds)',
    xscale='log',
    bounds=('38%', '66%', '5%', '85%'),
)
doPlot(axes23, '2-3 Swap')

axesRemainder = canvas.cartesian(
    xlabel = 'Number of Processes',
    #ylabel = 'Partial Composite Time (seconds)',
    xscale='log',
    bounds=('70%', '99%', '5%', '85%'),
)
doPlot(axesRemainder, 'Remainder')

<path d="M 44.400000000000006 115.59775680000001 L 84.422100756396816 27.858153599999984 L 101.51653882898884 26.243539200000008 L 117.12253808965417 29.134080000000012 L 144.77036391020133 28.780070400000003 L 168.72000000000003 117.41425919999998 L 179.59338042424218 32.457052800000042 L 208.74210075639684 32.068166399999967 L 225.83653882898889 34.859980799999981 L 248.76420151279379 34.177824000000022 L 269.09036391020135 34.524806400000045 L 293.04000000000002 120.0155616" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> <path d="M 44.400000000000006 147.16941120000001 L 64.275204588289412 62.200108799999995 L 85.539586287209133 65.373782400000039 L 106.54084068218413 62.750563199999995 L 127.11456122446116 60.293779200000017 L 147.94488656096078 61.99024320000003 L 168.72000000000003 148.87667519999999 L 189.22122912432155 66.172665599999988 L 209.85958628720914 60.146006399999983 L 230.86084068218412 55.905033599999996 L 251.43456122446116 57.501014400000031 L 272.26488656096058 59.56977599999999 L 293.04000000000002 148.45227839999998" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Virtual Node: Naive Pure Distributed: Naive 128 256 512 Number of Processes 0.00 0.01 0.02 0.03 0.04 Partial Composite Time (seconds) <path d="M 337.44 108.92108159999999 L 377.46210075639681 103.73552639999998 L 394.55653882898883 107.15823360000003 L 410.16253808965416 107.15973120000001 L 437.81036391020132 103.8425184 L 461.75999999999999 110.65268159999997 L 472.6333804242422 107.1341568 L 501.78210075639686 109.01738879999998 L 518.87653882898894 109.6956864 L 541.80420151279384 107.50541759999999 L 562.13036391020137 106.7617152 L 586.08000000000004 107.67689280000003" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> <path d="M 337.44 136.35207360000004 L 357.31520458828942 126.4254912 L 378.57958628720917 130.48493759999997 L 399.58084068218409 133.23453119999999 L 420.15456122446119 127.6016832 L 440.98488656096083 129.48652800000002 L 461.75999999999999 137.67966720000001 L 482.26122912432157 128.592288 L 502.89958628720916 132.6845088 L 523.90084068218414 134.47451520000001 L 544.47456122446113 127.78053119999996 L 565.30488656096054 131.700816 L 586.08000000000004 134.36746559999997" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Virtual Node: 2-3 Swap Pure Distributed: 2-3 Swap 128 256 512 Number of Processes 0.00 0.01 0.02 0.03 0.04 <path d="M 621.60000000000002 115.11881279999999 L 663.05146149769666 116.91264960000001 L 680.75641521573846 116.19135359999997 L 696.91977159285602 116.90297280000001 L 725.55501976413711 117.38488320000003 L 750.36000000000001 118.11821759999998 L 761.62171543939371 117.35611200000002 L 791.81146149769677 118.14995520000001 L 809.51641521573856 118.62408960000002 L 833.26292299539352 120.31752959999997 L 854.31501976413711 117.11744640000001 L 879.12 120.16670399999997" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> <path d="M 621.60000000000002 145.14048 L 642.18503332358546 144.25096319999997 L 664.20885722603805 145.8695232 L 685.96015642083353 146.52711360000001 L 707.26865269676341 145.5949152 L 728.84291822385228 145.97982719999999 L 750.36000000000001 147.79503360000001 L 771.59341587876156 145.94088960000005 L 792.96885722603804 147.37726079999999 L 814.72015642083352 146.83887360000003 L 836.0286526967634 148.23253440000002 L 857.60291822385204 148.3075584 L 879.12 148.29393600000003" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Virtual Node: Remainder Pure Distributed: Remainder 128 256 512 Number of Processes 0.00 0.01 0.02 0.03 0.04

In [14]:
toyplot.pdf.render(canvas, 'vn-vs-smp-hdtv.pdf')

In [15]:
image_size = '8K UHD'

canvas = toyplot.Canvas('9.25in', '3in',
                        #style={'background-color': 'yellow'},
                       )

x_vn = average_partial_composite_vn.index
x_smp = average_partial_composite_smp.index

maxy = max(
    numpy.max(numpy.max(average_partial_composite_vn[image_size])),
    numpy.max(numpy.max(average_partial_composite_smp[image_size])),
)

xlocator = toyplot.locator.Log(base=2, format='{:.0f}')

def doPlot(axes, algorithm):
    axes.x.ticks.locator = xlocator
    axes.y.domain.min = 0
    axes.y.domain.max = maxy
    
    y_vn = numpy.array(average_partial_composite_vn[image_size, algorithm])
    y_smp = numpy.array(average_partial_composite_smp[image_size, algorithm])

    axes.plot(x_vn, y_vn)
    axes.plot(x_smp, y_smp)

    axes.text(
        512, numpy.max(y_vn), 'Virtual Node: ' + algorithm,
        style={
            'text-anchor': 'end',
            '-toyplot-vertical-align': 'last-baseline',
        }
    )
    axes.text(
        512, numpy.min(y_smp), 'Pure Distributed: ' + algorithm,
        style={
            'text-anchor': 'end',
            '-toyplot-vertical-align': 'top',
        }
    )

axesNaive = canvas.cartesian(
    xlabel = 'Number of Processes',
    ylabel = 'Partial Composite Time (seconds)',
    xscale='log',
    bounds=('5%', '33%', '5%', '85%'),
)
doPlot(axesNaive, 'Naive')

axes23 = canvas.cartesian(
    xlabel = 'Number of Processes',
    #ylabel = 'Partial Composite Time (seconds)',
    xscale='log',
    bounds=('38%', '66%', '5%', '85%'),
)
doPlot(axes23, '2-3 Swap')

axesRemainder = canvas.cartesian(
    xlabel = 'Number of Processes',
    #ylabel = 'Partial Composite Time (seconds)',
    xscale='log',
    bounds=('70%', '99%', '5%', '85%'),
)
doPlot(axesRemainder, 'Remainder')

<path d="M 44.400000000000006 106.43367935999999 L 84.422100756396816 34.179333119999981 L 101.51653882898884 34.066237440000009 L 117.12253808965417 36.05644800000001 L 144.77036391020133 36.533913599999963 L 168.72000000000003 121.13719295999999 L 179.59338042424218 40.427903999999977 L 208.74210075639684 39.513123839999963 L 225.83653882898889 41.037696000000004 L 248.76420151279379 41.484288000000035 L 269.09036391020135 41.259448319999976 L 293.04000000000002 123.24559871999999" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> <path d="M 44.400000000000006 145.00078080000003 L 64.275204588289412 54.224010240000034 L 85.539586287209133 57.623700479999954 L 106.54084068218413 60.087060479999998 L 127.11456122446116 58.476518399999961 L 147.94488656096078 58.921528320000036 L 168.72000000000003 149.98156799999998 L 189.22122912432155 61.540346879999987 L 209.85958628720914 62.414146560000027 L 230.86084068218412 63.674680320000022 L 251.43456122446116 65.735608319999997 L 272.26488656096058 65.65470719999999 L 293.04000000000002 152.55905279999999" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Virtual Node: Naive Pure Distributed: Naive 128 256 512 Number of Processes 0.0 0.2 0.5 0.8 Partial Composite Time (seconds) <path d="M 337.44 114.27632639999996 L 377.46210075639681 109.46789376000002 L 394.55653882898883 108.41001983999999 L 410.16253808965416 110.21200895999999 L 437.81036391020132 110.35038720000001 L 461.75999999999999 117.60327167999996 L 472.6333804242422 113.16883968 L 501.78210075639686 113.81139455999997 L 518.87653882898894 113.56896767999999 L 541.80420151279384 114.00949247999998 L 562.13036391020137 115.20087552 L 586.08000000000004 119.60578559999999" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> <path d="M 337.44 138.45955583999998 L 357.31520458828942 127.035264 L 378.57958628720917 133.30705919999997 L 399.58084068218409 137.25709824 L 420.15456122446119 131.66466048000001 L 440.98488656096083 134.28326399999997 L 461.75999999999999 143.55019776000003 L 482.26122912432157 134.29495295999996 L 502.89958628720916 137.54781696000001 L 523.90084068218414 140.54261759999997 L 544.47456122446113 137.68982015999998 L 565.30488656096054 139.20422399999998 L 586.08000000000004 145.51606271999998" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Virtual Node: 2-3 Swap Pure Distributed: 2-3 Swap 128 256 512 Number of Processes 0.0 0.2 0.5 0.8 <path d="M 621.60000000000002 117.29925119999999 L 663.05146149769666 118.82861568000003 L 680.75641521573846 117.97005312 L 696.91977159285602 119.08483583999997 L 725.55501976413711 119.86008576000003 L 750.36000000000001 112.63351295999999 L 761.62171543939371 120.80252928000003 L 791.81146149769677 121.93881599999997 L 809.51641521573856 122.87688191999999 L 833.26292299539352 122.83244544000001 L 854.31501976413711 121.24388352 L 879.12 123.41524991999999" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> <path d="M 621.60000000000002 144.65530367999997 L 642.18503332358546 143.67094271999997 L 664.20885722603805 141.75988224 L 685.96015642083353 146.98321920000004 L 707.26865269676341 146.28264960000001 L 728.84291822385228 148.33121279999997 L 750.36000000000001 150.07635456000003 L 771.59341587876156 148.91318783999998 L 792.96885722603804 149.41335552000001 L 814.72015642083352 150.50451455999996 L 836.0286526967634 151.29987071999994 L 857.60291822385204 152.86447103999998 L 879.12 152.36089343999998" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Virtual Node: Remainder Pure Distributed: Remainder 128 256 512 Number of Processes 0.0 0.2 0.5 0.8

In [16]:
toyplot.pdf.render(canvas, 'vn-vs-smp-8k.pdf')